In [1]:
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

### Информация по рекламе

In [2]:
ads = pd.read_csv('Data_sources/ads.csv')
ads

,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,d_utm_term,m_clicks,m_cost
0,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8404700756,NaN,2.0,48.840
1,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476161,NaN,3.0,100.776
2,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476162,NaN,1.0,10.140
3,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476163,NaN,0.0,0.000
4,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476164,NaN,0.0,0.000
...,...,...,...,...,...,...,...,...,...
8597,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477628,NaN,0.0,0.000
8598,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477629,NaN,0.0,0.000
8599,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8404708323,NaN,0.0,0.000
8600,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8404708324,NaN,0.0,0.000


In [3]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8602 entries, 0 to 8601
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   created_at       8602 non-null   object 
 1   d_ad_account_id  8602 non-null   object 
 2   d_utm_source     8602 non-null   object 
 3   d_utm_medium     8602 non-null   object 
 4   d_utm_campaign   8602 non-null   int64  
 5   d_utm_content    8602 non-null   int64  
 6   d_utm_term       0 non-null      float64
 7   m_clicks         8602 non-null   float64
 8   m_cost           8602 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 605.0+ KB


In [4]:
ads.describe()

,d_utm_campaign,d_utm_content,d_utm_term,m_clicks,m_cost
count,8.602000e+03,8.602000e+03,0.0,8602.000000,8602.000000
mean,4.869139e+07,8.677256e+09,NaN,2.185189,52.446061
std,2.870346e+06,4.452434e+08,NaN,3.945861,102.918082
min,4.830644e+07,8.404701e+09,NaN,0.000000,0.000000
25%,4.830645e+07,8.404705e+09,NaN,0.000000,0.000000
50%,4.830649e+07,8.813433e+09,NaN,0.000000,0.000000
75%,4.830652e+07,8.813476e+09,NaN,3.000000,60.147000
max,7.200079e+07,1.184486e+10,NaN,39.000000,1103.616000


In [5]:
ads['created_at'] = pd.to_datetime(ads['created_at'], format='%Y-%m-%d')

In [6]:
ads.duplicated().sum()

0

In [7]:
for col in ads.columns:
    if ads[col].isna().sum() > 0:
        print(f'В колонке {col} - {ads[col].isna().sum()} пустых строк, \
что составляет {ads[col].isna().sum() / len(ads):.0%} данных')

В колонке d_utm_term - 8602 пустых строк, что составляет 100% данных


Столбец `d_utm_term` польностью состоит из пропусков

In [8]:
ads.drop(columns='d_utm_term', inplace=True)

In [9]:
for col in ads.columns:
    if ads[col].dtypes == 'object':
        print(ads[col].value_counts(normalize=True))
        print()

xo-for-client-ya    1.0
Name: d_ad_account_id, dtype: float64

yandex    1.0
Name: d_utm_source, dtype: float64

cpc    1.0
Name: d_utm_medium, dtype: float64



Представлена информация только по одному источнику и каналу

Для возможности связать с таблицей leads изменим типы некоторых столбцов

In [10]:
ads['d_utm_campaign'] = ads['d_utm_campaign'].astype('str')
ads['d_utm_content'] = ads['d_utm_content'].astype('str')

### Информация по лидам

In [11]:
leads=pd.read_csv('Data_sources/leads.csv')
leads.sample(3)

,lead_created_at,lead_id,d_lead_utm_source,d_lead_utm_medium,d_lead_utm_campaign,d_lead_utm_content,d_lead_utm_term,client_id
4265,2022-07-29,557b09d5-0f5d-11ed-9074-848f69e142cf,razdatka,cpa,31-07,NaN,NaN,d07cf03a-0fd3-11ed-9074-848f69e142cf
14976,2022-01-11,cd47ebc1-7305-11ec-9057-848f69e142cf,instagram,cpc,23849173065740725,23849173065730725,NaN,23d33610-bdf5-11eb-ac38-c412f533dba1
8454,2022-04-02,beb77958-b281-11ec-9061-848f69e142cf,eLama-google,cpc,Поиск II Брендовые запросы,cid|8655725448|gid|87321288232|aid|40696847899...,+физкультура +фитнес +самара,63843bef-3fec-11ec-904f-848f69e142cf


In [12]:
leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23540 entries, 0 to 23539
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   lead_created_at      23540 non-null  object
 1   lead_id              23540 non-null  object
 2   d_lead_utm_source    12331 non-null  object
 3   d_lead_utm_medium    10100 non-null  object
 4   d_lead_utm_campaign  9569 non-null   object
 5   d_lead_utm_content   3338 non-null   object
 6   d_lead_utm_term      328 non-null    object
 7   client_id            18601 non-null  object
dtypes: object(8)
memory usage: 1.4+ MB


In [13]:
leads['lead_created_at'] = pd.to_datetime(leads['lead_created_at'], format='%Y-%m-%d')


In [14]:
for col in ['d_lead_utm_source', 'd_lead_utm_medium']:
    if leads[col].dtypes == 'object':
        print(leads[col].value_counts(normalize=True))
        print()

VOICEROBOT                                  0.178412
vk                                          0.166329
instagram                                   0.152380
yandex                                      0.145892
sms                                         0.087179
vkontakte                                   0.054254
ycard                                       0.027248
eLama-google                                0.025545
megacity                                    0.023275
ff110922                                    0.020031
df310722                                    0.016219
zvonobot                                    0.015246
kviz                                        0.013624
vkmegacity                                  0.012975
telegram                                    0.009002
outdoor                                     0.007704
dz1809                                      0.007136
clients_day                                 0.005271
promo7                                      0.

Интересующему нас источнику соответствует `yandex` соответствует 14,5% записей в таблице заявок.

In [15]:
for col in leads.columns:
    if leads[col].isna().sum() > 0:
        print(f'В колонке {col} - {leads[col].isna().sum()} пустых строк, \
что составляет {leads[col].isna().sum() / len(leads):.0%} данных')

В колонке d_lead_utm_source - 11209 пустых строк, что составляет 48% данных
В колонке d_lead_utm_medium - 13440 пустых строк, что составляет 57% данных
В колонке d_lead_utm_campaign - 13971 пустых строк, что составляет 59% данных
В колонке d_lead_utm_content - 20202 пустых строк, что составляет 86% данных
В колонке d_lead_utm_term - 23212 пустых строк, что составляет 99% данных
В колонке client_id - 4939 пустых строк, что составляет 21% данных


Столбец `d_lead_utm_term ` содержит 99% пропусков, остальные столбцы, необходимые для связей с таблицами ads и purchases также содержат пропуски. Данные пропуски могут быть удалены, а столбец `d_lead_utm_term` удален полностью, т.к. ранее он был удален и из таблицы `ads`

In [16]:
leads.drop(columns='d_lead_utm_term', inplace=True)

In [17]:
leads.dropna(subset=['d_lead_utm_source', 'd_lead_utm_medium', 'd_lead_utm_campaign', 'd_lead_utm_content', \
                    'client_id'], how='any', inplace=True)

In [18]:
leads.duplicated().sum()

0

Связь с таблицей ads определяется набором столбцов с датой и utm-метками. При наличии дублей данная связь не будет однозначной и, соответственно, увеличится кол-во кликов и их суммарная стоимость.

In [19]:
leads[(leads['d_lead_utm_source']=='yandex')&(leads['d_lead_utm_medium']=='cpc')][['lead_created_at', \
'd_lead_utm_source', 'd_lead_utm_medium', 'd_lead_utm_campaign', 'd_lead_utm_content']].duplicated().sum()

338

Данная связь будет характеризоваться как один(ads) ко многим(leads) и, соответственно, потребуется дополнительные преобразования для исключения дублирования кол-ва кликов и суммы расходов на рекламу.

In [20]:
ads['m_clicks'].sum()

18797.0

In [21]:
total = ads.merge(leads, how='left', 
                        left_on=['created_at', 'd_utm_source', 'd_utm_medium', 'd_utm_campaign', 'd_utm_content'],
                        right_on=['lead_created_at', 'd_lead_utm_source', 'd_lead_utm_medium', 
                                        'd_lead_utm_campaign', 'd_lead_utm_content', ])

In [22]:
total['m_clicks'].sum()

22471.0

Кол-во кликов выросло за счет наличия дублирующихся строк в таблице с лидами

Удалим лишние столбцы

In [23]:
total.drop(columns=['d_ad_account_id','lead_id', 'd_lead_utm_source', 'd_lead_utm_medium', \
                    'd_lead_utm_campaign', 'd_lead_utm_content'], inplace=True)

Посчитаем количество записей для каждого уникального набора ключей, сформированных по набору полей `created_at`, `d_utm_source`, `d_utm_medium`, `d_utm_campaign`, `d_utm_content`.

In [24]:
total['count'] = total.groupby(by=['created_at','d_utm_source','d_utm_medium','d_utm_campaign', \
                                    'd_utm_content'])['m_cost'].transform(lambda x: x.count())

In [25]:
total['count'].value_counts()

1    8396
2     290
3     144
4      44
5      10
Name: count, dtype: int64

В большинстве случаев ключи уникальны, двойные дубли встречаются 290 раз, максимальное количество повторов составляет пять строк.

Восстановим правильное количество кликов и расходов на рекламу, разделив на количество дублирующихся записей по каждому ключу. Т.е. при наличии нескольких лидов с совпадающим набором ключей из даты и utm-меток, будем считать, что каждому лиду относится пропорциональное количество кликов.

In [26]:
total['m_clicks_per_lead'] = total['m_clicks'] / total['count']
total['m_cost_per_lead'] = total['m_cost'] / total['count']

In [27]:
total['m_clicks_per_lead'].sum()

18797.0

### Информация по покупкам

In [28]:
purchases=pd.read_csv('Data_sources/purchases.csv')
purchases.head()

,purchase_created_at,purchase_id,client_id,m_purchase_amount
0,2022-02-21,d3198d39-6d16-40c9-bff7-aa28bd6e2991,7011bdcd-6fd8-11e7-80fc-c412f533dba1,9950.0
1,2022-02-21,48a0ad24-77aa-4064-a971-dd0d6f1f6c50,a2771bb0-6fd6-11e7-80fc-c412f533dba1,8700.0
2,2022-02-21,267ff20f-a56c-480a-b2f7-1f7b1a746f9f,2687f503-6fd7-11e7-80fc-c412f533dba1,0.0
3,2022-02-21,19fba5f0-785d-45bd-903a-34ba529c6404,01122a97-45bc-11eb-ac23-c412f533dba1,8700.0
4,2022-02-21,00ac921d-32e3-4200-9435-a46d5fef4a29,a55b8c4e-d5a3-11e9-abbe-c412f533dba1,500.0


In [29]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66180 entries, 0 to 66179
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   purchase_created_at  66180 non-null  object 
 1   purchase_id          66180 non-null  object 
 2   client_id            66179 non-null  object 
 3   m_purchase_amount    66180 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


In [30]:
purchases.dropna(subset=['client_id'], how='any', inplace=True)

In [31]:
purchases['purchase_created_at'] = pd.to_datetime(purchases['purchase_created_at'], format='%Y-%m-%d')


In [32]:
for col in purchases.columns:
    if purchases[col].isna().sum() > 0:
        print(f'В колонке {col} - {purchases[col].isna().sum()} пустых строк, \
что составляет {purchases[col].isna().sum() / len(purchases):.0%} данных')

In [33]:
purchases.duplicated().sum()

0

In [34]:
len(purchases[purchases['m_purchase_amount'] == 0])

8663

Большое количество записей в таблице с покупками содержит нулевую сумму, что, вероятно, должно быть интерпретировано как отсутствие покупки и не учитываться в суммарном их количестве.

In [35]:
purchases = purchases.drop(purchases[purchases['m_purchase_amount'] == 0].index)

In [36]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57516 entries, 0 to 66179
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   purchase_created_at  57516 non-null  datetime64[ns]
 1   purchase_id          57516 non-null  object        
 2   client_id            57516 non-null  object        
 3   m_purchase_amount    57516 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.2+ MB


### Объединение таблиц

Для каждого покупателя ('client_id') найдем все даты по лидам и, соответствующие этим лидам покупки, если они имеются.

In [37]:
leads_gr = leads.groupby('client_id', as_index=False).agg({'lead_created_at':set})
leads_gr

,client_id,lead_created_at
0,001f4c39-7a96-11ec-9058-848f69e142c9,{2022-01-21 00:00:00}
1,005795d3-744c-11ec-9057-848f69e142cf,{2022-01-13 00:00:00}
2,0070edee-2501-11ed-9078-848f69e142cf,{2022-08-25 00:00:00}
3,00817883-6fd6-11e7-80fc-c412f533dba1,{2022-01-31 00:00:00}
4,00b00fbc-80aa-11eb-ac27-c412f533dba1,{2022-09-04 00:00:00}
...,...,...
2473,ff6fe041-340d-11ed-907c-848f69e142cf,{2022-10-05 00:00:00}
2474,ff7d081f-d070-11ec-9066-848f69e142cd,{2022-05-10 00:00:00}
2475,ff9e6518-ec9d-11ec-906f-848f69e142cf,{2022-06-15 00:00:00}
2476,ffcd029e-7dad-11ec-9059-848f69e142c9,{2022-01-25 00:00:00}


In [38]:
pur_gr = purchases.groupby('client_id', as_index=False).agg({'purchase_created_at':set})
pur_gr

,client_id,purchase_created_at
0,000ed948-63bd-11eb-ac26-c412f533dba1,"{2022-02-04 00:00:00, 2022-07-01 00:00:00}"
1,00104ea2-d85b-11ec-9067-848f69e142c9,{2022-05-23 00:00:00}
2,00180c2e-9155-11e7-8114-c412f533dba1,"{2022-08-11 00:00:00, 2022-05-30 00:00:00, 202..."
3,001f4c39-7a96-11ec-9058-848f69e142c9,"{2022-02-20 00:00:00, 2022-03-16 00:00:00, 202..."
4,002152fc-ffdb-11ea-ac18-c412f533dba1,"{2022-09-22 00:00:00, 2022-03-14 00:00:00, 202..."
...,...,...
15281,ffdb8f20-b7fa-11ec-9062-848f69e142cf,"{2022-04-28 00:00:00, 2022-04-14 00:00:00}"
15282,ffe3d29d-2c48-11e8-8131-c412f533dba1,{2022-05-16 00:00:00}
15283,ffe42035-52c8-11ec-9052-848f69e142cf,{2022-09-29 00:00:00}
15284,ffe76cba-f12d-11eb-903e-848f69e142cd,{2022-04-25 00:00:00}


In [39]:
ld_pr = leads_gr.merge(pur_gr, how='left', on='client_id')
ld_pr

,client_id,lead_created_at,purchase_created_at
0,001f4c39-7a96-11ec-9058-848f69e142c9,{2022-01-21 00:00:00},"{2022-02-20 00:00:00, 2022-03-16 00:00:00, 202..."
1,005795d3-744c-11ec-9057-848f69e142cf,{2022-01-13 00:00:00},NaN
2,0070edee-2501-11ed-9078-848f69e142cf,{2022-08-25 00:00:00},NaN
3,00817883-6fd6-11e7-80fc-c412f533dba1,{2022-01-31 00:00:00},{2022-01-31 00:00:00}
4,00b00fbc-80aa-11eb-ac27-c412f533dba1,{2022-09-04 00:00:00},NaN
...,...,...,...
2473,ff6fe041-340d-11ed-907c-848f69e142cf,{2022-10-05 00:00:00},NaN
2474,ff7d081f-d070-11ec-9066-848f69e142cd,{2022-05-10 00:00:00},NaN
2475,ff9e6518-ec9d-11ec-906f-848f69e142cf,{2022-06-15 00:00:00},NaN
2476,ffcd029e-7dad-11ec-9059-848f69e142c9,{2022-01-25 00:00:00},NaN


In [40]:
ld_pr['lead_created_at'] = ld_pr['lead_created_at'].apply(lambda x: sorted(list(x)))
ld_pr['purchase_created_at'] = ld_pr['purchase_created_at'].apply(lambda x: sorted(list(x)) if pd.notna(x) else x)

In [41]:
ld_pr[120:130]

,client_id,lead_created_at,purchase_created_at
120,0c6eba7c-1b64-11ec-9045-848f69e142c9,[2022-02-15 00:00:00],[2022-01-16 00:00:00]
121,0c71bc66-2c32-11ed-907b-848f69e142cf,[2022-09-04 00:00:00],NaN
122,0c8a4706-78f8-11ec-9058-848f69e142c9,[2022-01-19 00:00:00],NaN
123,0c98c6a8-615c-11ec-9054-848f69e142cf,[2022-01-19 00:00:00],NaN
124,0d1a0d65-cb39-11ea-ac0f-c412f533dba1,[2022-03-22 00:00:00],[2022-03-22 00:00:00]
125,0d1a9ac8-b25b-11ec-9061-848f69e142cf,[2022-04-02 00:00:00],NaN
126,0d2130a5-2453-11ed-9078-848f69e142cf,"[2022-08-24 00:00:00, 2022-09-15 00:00:00]","[2022-09-19 00:00:00, 2022-09-21 00:00:00]"
127,0d29c4a9-d5ab-11e9-abbe-c412f533dba1,[2022-05-06 00:00:00],NaN
128,0d382de0-011b-11ed-9072-848f69e142cf,[2022-07-11 00:00:00],NaN
129,0d410c70-7f3e-11ec-9059-848f69e142c9,[2022-01-27 00:00:00],NaN


Отделим на время, для удобства последующих преобразований, клиентов без покупок.

In [42]:
ld_pr_notna = ld_pr[pd.notna(ld_pr['purchase_created_at'])]
ld_pr_isna = ld_pr[pd.isna(ld_pr['purchase_created_at'])]
ld_pr_notna

,client_id,lead_created_at,purchase_created_at
0,001f4c39-7a96-11ec-9058-848f69e142c9,[2022-01-21 00:00:00],"[2022-02-20 00:00:00, 2022-03-16 00:00:00, 202..."
3,00817883-6fd6-11e7-80fc-c412f533dba1,[2022-01-31 00:00:00],[2022-01-31 00:00:00]
7,01031b3a-b59a-11e8-ab73-c412f533dba1,[2022-01-23 00:00:00],"[2022-01-24 00:00:00, 2022-02-08 00:00:00, 202..."
8,0123c572-adb4-11e7-8116-c412f533dba1,[2022-01-21 00:00:00],[2022-04-26 00:00:00]
10,01a36ca5-1173-11ed-9074-848f69e142cf,[2022-08-01 00:00:00],[2022-08-01 00:00:00]
...,...,...,...
2464,fe70d078-98b7-11e9-abac-c412f533dba1,[2022-08-26 00:00:00],[2022-02-19 00:00:00]
2465,fe77adfa-f9e4-11eb-903f-848f69e142cf,[2022-06-26 00:00:00],[2022-06-28 00:00:00]
2467,fea10060-b314-11ec-9062-848f69e142cf,[2022-04-18 00:00:00],[2022-04-29 00:00:00]
2468,fea18e6f-7eae-11ec-9059-848f69e142c9,[2022-01-04 00:00:00],"[2022-01-26 00:00:00, 2022-02-27 00:00:00, 202..."


Определим функцию, для однозначной связи лидов с покупками, согласно выданным условиям. Если нет соответствующей покупки, для приведения всех данных к одному типубудем выставлять отдаленную дату

In [43]:
def corr_purchase_date(lead_dt, purchase_dt):
    '''
    Ф-я принимает на вход список дат по заявкам и список дат по покупкам,
    возвращает словарь, в котором каждой дате заявки (ключу) ставится в соответствие дата покупки (значение)
    Если нет соответствующей покупки, то в качестве значения возвращает '2099-01-01'.
    '''
    ld_pr_dct = {}
    for ld_ind, ld_dt in enumerate(lead_dt):
        for pr_ind, pr_dt in enumerate(purchase_dt):
            if  ld_dt not in ld_pr_dct:
                if pd.Timedelta('0 days') <= pr_dt - ld_dt  <= pd.Timedelta('15 days'):
                    if ld_dt == lead_dt[-1]:
                        ld_pr_dct[ld_dt] = pr_dt
                    elif pr_dt - lead_dt[ld_ind+1]  < pd.Timedelta('0 days'):
                        ld_pr_dct[ld_dt] = pr_dt
                    else:
                        ld_pr_dct[ld_dt] = pd.to_datetime('2099-01-01')
        if  ld_dt not in ld_pr_dct:
            ld_pr_dct[ld_dt] = pd.to_datetime('2099-01-01')

    return ld_pr_dct

Проверим функцию на корректность работы, подав на вход несколько возможных случаев соотношения даты оформления заявок и даты покупок

In [44]:
lead_dt = [pd.to_datetime('2022-01-21')]
purchase_dt = [pd.to_datetime('2022-01-25'), pd.to_datetime('2022-02-27')]
corr_purchase_date(lead_dt, purchase_dt)

{Timestamp('2022-01-21 00:00:00'): Timestamp('2022-01-25 00:00:00')}

In [45]:
lead_dt = [pd.to_datetime('2022-01-21')]
purchase_dt = [pd.to_datetime('2022-02-15'), pd.to_datetime('2022-03-27')]
corr_purchase_date(lead_dt, purchase_dt)

{Timestamp('2022-01-21 00:00:00'): Timestamp('2099-01-01 00:00:00')}

In [46]:
lead_dt = [pd.to_datetime('2022-01-21'), pd.to_datetime('2022-05-21'), pd.to_datetime('2022-06-22')]
purchase_dt = [pd.to_datetime('2022-01-20'), pd.to_datetime('2022-02-01'), pd.to_datetime('2022-06-25')]
corr_purchase_date(lead_dt, purchase_dt)

{Timestamp('2022-01-21 00:00:00'): Timestamp('2022-02-01 00:00:00'),
 Timestamp('2022-05-21 00:00:00'): Timestamp('2099-01-01 00:00:00'),
 Timestamp('2022-06-22 00:00:00'): Timestamp('2022-06-25 00:00:00')}

In [47]:
lead_dt = [pd.to_datetime('2022-01-21'), pd.to_datetime('2022-03-15'), pd.to_datetime('2022-04-15')]
purchase_dt = [pd.to_datetime('2022-01-25'), pd.to_datetime('2022-02-23'), pd.to_datetime('2022-03-23')]
corr_purchase_date(lead_dt, purchase_dt)

{Timestamp('2022-01-21 00:00:00'): Timestamp('2022-01-25 00:00:00'),
 Timestamp('2022-03-15 00:00:00'): Timestamp('2022-03-23 00:00:00'),
 Timestamp('2022-04-15 00:00:00'): Timestamp('2099-01-01 00:00:00')}

In [48]:
ld_pr_notna['lead_purchase'] = ld_pr_notna.apply(lambda x: corr_purchase_date(x.lead_created_at, \
                                                    x.purchase_created_at), axis=1)

In [49]:
ld_pr_notna

,client_id,lead_created_at,purchase_created_at,lead_purchase
0,001f4c39-7a96-11ec-9058-848f69e142c9,[2022-01-21 00:00:00],"[2022-02-20 00:00:00, 2022-03-16 00:00:00, 202...",{2022-01-21 00:00:00: 2099-01-01 00:00:00}
3,00817883-6fd6-11e7-80fc-c412f533dba1,[2022-01-31 00:00:00],[2022-01-31 00:00:00],{2022-01-31 00:00:00: 2022-01-31 00:00:00}
7,01031b3a-b59a-11e8-ab73-c412f533dba1,[2022-01-23 00:00:00],"[2022-01-24 00:00:00, 2022-02-08 00:00:00, 202...",{2022-01-23 00:00:00: 2022-01-24 00:00:00}
8,0123c572-adb4-11e7-8116-c412f533dba1,[2022-01-21 00:00:00],[2022-04-26 00:00:00],{2022-01-21 00:00:00: 2099-01-01 00:00:00}
10,01a36ca5-1173-11ed-9074-848f69e142cf,[2022-08-01 00:00:00],[2022-08-01 00:00:00],{2022-08-01 00:00:00: 2022-08-01 00:00:00}
...,...,...,...,...
2464,fe70d078-98b7-11e9-abac-c412f533dba1,[2022-08-26 00:00:00],[2022-02-19 00:00:00],{2022-08-26 00:00:00: 2099-01-01 00:00:00}
2465,fe77adfa-f9e4-11eb-903f-848f69e142cf,[2022-06-26 00:00:00],[2022-06-28 00:00:00],{2022-06-26 00:00:00: 2022-06-28 00:00:00}
2467,fea10060-b314-11ec-9062-848f69e142cf,[2022-04-18 00:00:00],[2022-04-29 00:00:00],{2022-04-18 00:00:00: 2022-04-29 00:00:00}
2468,fea18e6f-7eae-11ec-9059-848f69e142c9,[2022-01-04 00:00:00],"[2022-01-26 00:00:00, 2022-02-27 00:00:00, 202...",{2022-01-04 00:00:00: 2099-01-01 00:00:00}


Для каждого клиента имеем в столбце `lead_purchase` словарь, содержащий заявки и соответствующие им покупки. 

Распакуем каждое значение лида в отдельную строку и сформируем столбец с соответствующим значением покупки.

In [50]:
ld_pr_notna = ld_pr_notna.explode('lead_created_at')
ld_pr_notna['purchase_created_at'] = ld_pr_notna.apply(lambda x: x.lead_purchase.get(x.lead_created_at), axis=1)
ld_pr_notna.drop(columns='lead_purchase', inplace=True)
ld_pr_notna

,client_id,lead_created_at,purchase_created_at
0,001f4c39-7a96-11ec-9058-848f69e142c9,2022-01-21,2099-01-01
3,00817883-6fd6-11e7-80fc-c412f533dba1,2022-01-31,2022-01-31
7,01031b3a-b59a-11e8-ab73-c412f533dba1,2022-01-23,2022-01-24
8,0123c572-adb4-11e7-8116-c412f533dba1,2022-01-21,2099-01-01
10,01a36ca5-1173-11ed-9074-848f69e142cf,2022-08-01,2022-08-01
...,...,...,...
2464,fe70d078-98b7-11e9-abac-c412f533dba1,2022-08-26,2099-01-01
2465,fe77adfa-f9e4-11eb-903f-848f69e142cf,2022-06-26,2022-06-28
2467,fea10060-b314-11ec-9062-848f69e142cf,2022-04-18,2022-04-29
2468,fea18e6f-7eae-11ec-9059-848f69e142c9,2022-01-04,2099-01-01


In [51]:
ld_pr = pd.concat([ld_pr_notna, ld_pr_isna.fillna(pd.to_datetime('2099-01-01')).explode('lead_created_at')])

In [52]:
ld_pr

,client_id,lead_created_at,purchase_created_at
0,001f4c39-7a96-11ec-9058-848f69e142c9,2022-01-21,2099-01-01
3,00817883-6fd6-11e7-80fc-c412f533dba1,2022-01-31,2022-01-31
7,01031b3a-b59a-11e8-ab73-c412f533dba1,2022-01-23,2022-01-24
8,0123c572-adb4-11e7-8116-c412f533dba1,2022-01-21,2099-01-01
10,01a36ca5-1173-11ed-9074-848f69e142cf,2022-08-01,2022-08-01
...,...,...,...
2473,ff6fe041-340d-11ed-907c-848f69e142cf,2022-10-05,2099-01-01
2474,ff7d081f-d070-11ec-9066-848f69e142cd,2022-05-10,2099-01-01
2475,ff9e6518-ec9d-11ec-906f-848f69e142cf,2022-06-15,2099-01-01
2476,ffcd029e-7dad-11ec-9059-848f69e142c9,2022-01-25,2099-01-01


Проверим однозначность связей с ранее сформированной таблицей `total`

In [53]:
ld_pr[['client_id', 'lead_created_at']].value_counts()[:5]

client_id                             lead_created_at
001f4c39-7a96-11ec-9058-848f69e142c9  2022-01-21         1
ad3b5807-bcdd-11e8-ab73-c412f533dba1  2022-04-07         1
a7dcc843-4f57-11eb-ac23-c412f533dba1  2022-02-01         1
a7e0cada-e95b-11ec-906f-848f69e142cf  2022-06-11         1
a7e27568-965b-11ec-905d-848f69e142cf  2022-02-25         1
dtype: int64

In [54]:
total[['client_id', 'lead_created_at']].value_counts()[:10]

client_id                             lead_created_at
9eb261e7-49ef-11ec-9052-848f69e142cf  2022-01-06         2
1793491e-da65-11ec-9067-848f69e142c9  2022-05-22         2
85457286-1888-11e8-812f-c412f533dba1  2022-06-19         2
209ea53a-b708-11ec-9062-848f69e142cf  2022-04-08         2
77ee4152-26ca-11ed-907a-848f69e142cf  2022-08-28         2
e565e53e-c54d-11ec-9065-848f69e142cf  2022-04-26         2
4bd33687-d5ac-11e9-abbe-c412f533dba1  2022-06-27         2
5041b2a0-ce63-11e9-abbc-c412f533dba1  2022-06-20         2
0470fd1e-656a-11eb-ac26-c412f533dba1  2022-05-10         2
a65ffd34-b0b8-11ec-9061-848f69e142cf  2022-03-30         1
dtype: int64

Несколько записей имеют одинаковый набор полей по номеру клиента и даты формирования заявки

In [55]:
total =  total.merge(ld_pr, on=['client_id', 'lead_created_at'], how='left')

Количество лидов для данной записи определим как 1/a, где a - количество записей с совпадающими значениями `client_id`, `lead_created_at`. Т.е. если таких строк две, то для каждой строки кол-во лидов будет определено как 0,5, а сумма даст единицу.

In [56]:
total['count_lead'] = total.groupby(by=['client_id', \
                                        'lead_created_at'])['client_id'].transform(lambda x: 1/x.count())

Дополним таблицу информацией о сумме покупок

In [57]:
total =  total.merge(purchases, on=['client_id', 'purchase_created_at'], how='left')

По аналогии с кликами, для обработки неуникальных записей по полям `client_id`, `purchase_created_at`, рассчитаем кол-во покупок.

In [58]:
total['count_purch'] = total.groupby(by=['client_id', \
                                'purchase_created_at'])['m_purchase_amount'].transform(lambda x: x.count())

In [59]:
total['purchase_sum'] = total['m_purchase_amount'] / total['count_purch']

In [60]:
total['count'] = total.groupby(by=['created_at','d_utm_source','d_utm_medium','d_utm_campaign', \
                                    'd_utm_content'])['m_cost'].transform(lambda x: x.count())
total['m_clicks_per_lead'] = total['m_clicks'] / total['count']
total['m_cost_per_lead'] = total['m_cost'] / total['count']

In [61]:
total.fillna(0, inplace=True)

Определим функцию для расчета количества ненулевых значений в массиве.

In [62]:
def count_if_more_zero(x):
    cnt_lst = []
    for i in x:
        if i != 0:
            cnt_lst.append(i)
    return len(cnt_lst)

In [63]:
count_if_more_zero([0,1,2,3,2,1,0])

5

In [64]:
final_df = total.groupby(['created_at', 'd_utm_source', 'd_utm_medium', 'd_utm_campaign'], as_index=False) \
            .agg({'m_clicks_per_lead':'sum', 'm_cost_per_lead':'sum', 'count_lead':[count_if_more_zero],
                  'purchase_sum':[count_if_more_zero,'sum']})

In [65]:
final_df.columns=['Дата', 'UTM source', 'UTM medium', 'UTM campaign', 'Количество кликов', \
    'Расходы на рекламу', 'Количество лидов', 'Количество покупок', 'Выручка от продаж']

In [66]:
final_df

,Дата,UTM source,UTM medium,UTM campaign,Количество кликов,Расходы на рекламу,Количество лидов,Количество покупок,Выручка от продаж
0,2022-01-03,yandex,cpc,48306435,1.0,11.952,0,0,0.0
1,2022-01-03,yandex,cpc,48306450,10.0,190.128,0,0,0.0
2,2022-01-04,yandex,cpc,48306435,1.0,11.364,0,0,0.0
3,2022-01-04,yandex,cpc,48306450,29.0,617.616,1,1,290.0
4,2022-01-05,yandex,cpc,48306435,0.0,0.000,0,0,0.0
...,...,...,...,...,...,...,...,...,...
1277,2022-09-14,yandex,cpc,48306450,48.0,1856.208,3,1,10400.0
1278,2022-09-14,yandex,cpc,48306473,1.0,33.444,0,0,0.0
1279,2022-09-14,yandex,cpc,48306487,10.0,228.060,0,0,0.0
1280,2022-09-14,yandex,cpc,48306494,0.0,0.000,0,0,0.0


In [67]:
final_df['CPL'] = final_df['Расходы на рекламу'] / final_df['Количество лидов']
final_df['ROAS'] = final_df['Выручка от продаж'] / final_df['Расходы на рекламу']
final_df.fillna(0, inplace=True)
final_df.replace(np.inf, '-', inplace=True)


In [68]:
final_df

,Дата,UTM source,UTM medium,UTM campaign,Количество кликов,Расходы на рекламу,Количество лидов,Количество покупок,Выручка от продаж,CPL,ROAS
0,2022-01-03,yandex,cpc,48306435,1.0,11.952,0,0,0.0,-,0.0
1,2022-01-03,yandex,cpc,48306450,10.0,190.128,0,0,0.0,-,0.0
2,2022-01-04,yandex,cpc,48306435,1.0,11.364,0,0,0.0,-,0.0
3,2022-01-04,yandex,cpc,48306450,29.0,617.616,1,1,290.0,617.616,0.469547
4,2022-01-05,yandex,cpc,48306435,0.0,0.000,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1277,2022-09-14,yandex,cpc,48306450,48.0,1856.208,3,1,10400.0,618.736,5.60282
1278,2022-09-14,yandex,cpc,48306473,1.0,33.444,0,0,0.0,-,0.0
1279,2022-09-14,yandex,cpc,48306487,10.0,228.060,0,0,0.0,-,0.0
1280,2022-09-14,yandex,cpc,48306494,0.0,0.000,0,0,0.0,0.0,0.0


In [69]:
final_df[final_df['Дата']==pd.to_datetime('2022-05-30')]

,Дата,UTM source,UTM medium,UTM campaign,Количество кликов,Расходы на рекламу,Количество лидов,Количество покупок,Выручка от продаж,CPL,ROAS
657,2022-05-30,yandex,cpc,48306435,10.0,300.744,1,0,0.0,300.744,0.0
658,2022-05-30,yandex,cpc,48306450,36.0,1100.700,4,3,25000.0,275.175,22.712819
659,2022-05-30,yandex,cpc,48306473,1.0,11.040,0,0,0.0,-,0.0
660,2022-05-30,yandex,cpc,48306487,1.0,8.112,0,0,0.0,-,0.0
661,2022-05-30,yandex,cpc,48306494,15.0,277.452,5,0,0.0,55.4904,0.0
662,2022-05-30,yandex,cpc,48306518,2.0,52.668,1,0,0.0,52.668,0.0


In [71]:
final_df.to_excel('Report/report.xlsx')